# Theory
In this PLN tutorial, we are going to make our own (the reason to call this tutorial  "_by Hand_") functions to use them for the purpose of '_relationship deduction on atoms of an atomspace_' and '_assertion on the deductions_' rather than using the already existing opencog modules made for the same purpose. These functions are defined and used in three scheme scripts, _deduction.scm_, _formula.scm_ and _simple-assertions.scm_.
#### formula.scm
This scheme script contains functions for performing mathimaics of various probability theories.
#### deduction.scm
This script deduces new fact based on the given relationship of the atoms in the atomspace and then initializes the atoms with their corresponding newly deduced relationship(inheritance), if any relation is actually deduced. It uses functions from _formula.scm_ to make deductions based on probability theories.
#### simple-assertions.scm
This script does two things. First, it initializes ConceptNode atoms in the atomspace with some relationship among them using the InheritanceLink link type. Then it uses BindLink to define a function named _find-humans_ that will look for all atoms in the atomspace inheriting the ConceptNode 'human' and then initializes ListLink with these atoms. So, what the _find-humans_ function actually do is make an assertion if a relation to 'human' is deduced.


# Introduction 
In this tutorial there are two basic sections:
> [For Notebook Environment](http://localhost:8888/notebooks/cloned/opencog_notebooks/PLN%20by%20hand.ipynb#For-Notebook-Environment)<br>
> [For scheme Environment](http://localhost:8888/notebooks/cloned/opencog_notebooks/PLN%20by%20hand.ipynb#For-scheme-Environment)

Both sections are the same in their content except that for the notebook Environment section, each function and node is loaded into the notebook environment by manually passing the corresponding code segment rather than using a script file, which is unlike for the scheme environment where scripts are used to load the functions and nodes to be used in the tutorial.

The remaining two sections are :
> [additional practice](http://localhost:8888/notebooks/cloned/opencog_notebooks/PLN%20by%20hand.ipynb#additional-practice)<br>
**Contains a sample code where the learner can modifiy and practice using their own examples**<br>
> [More PLN Examples](http://localhost:8888/notebooks/cloned/opencog_notebooks/PLN%20by%20hand.ipynb#More-PLN-Examples)<br>
**Contains paths for additional examples from the local opencog repository where the learner can further practice on**

# Initial setup
This tutorial can be followed on scheme environment or <br>
You can run the codes in the cells inside the notebook, if you have installed [Guile kernel](https://github.com/jerry40/guile-kernel), rather than copy pasting to the scheme environment.

# For Notebook Environment

**_load required modules_**

In [1]:
(use-modules (ice-9 readline)) 
(activate-readline)
(add-to-load-path "/usr/local/share/opencog/scm")
(add-to-load-path ".")
(use-modules (opencog))
(use-modules (opencog query))
(use-modules (opencog exec))
(use-modules (opencog logger))

#<unspecified>

**_load the code of formula.scm_**

In [2]:
;; =============================================================================
;; A list of the helper functions necessary in different inference rules
;; -----------------------------------------------------------------------------
;; Index
;; - simple-deduction-strength-formula
;; - inversion-strength-formula
;;------------------------------------------------------------------------------

;; =============================================================================
;; Simple Deduction Formula
;;
;; Preconditions:
;;
;;   1. Check that P(B|A) makes sense
;;
;;      1.a. P(B|A) is defined
;;
;;         0.0 < sA
;;
;;      1.b. P(B|A) is greater than or equal to P(A,B)/P(A) considering
;;      the smallest possible intersection between A and B
;;
;;         max((sA+sB-1)/sA, 0) <= sAB
;;
;;      1.c. P(B|A) is smaller than P(A,B)/P(A) considering the largest
;;      possible intersection between A and B
;;
;;         sAB <= min(1, sB/sA)
;;
;;   2. Check that P(C|B) makes sense
;;
;;      1.a. P(C|B) is defined
;;
;;         0.0 < sB
;;
;;      1.b. P(C|B) is greater than P(B,C)/P(B) considering the
;;      smallest possible intersection between B and C
;;
;;         max((sB+sC-1)/sB, 0) <= sBC
;;
;;      1.c. P(C|B) is greater than P(B,C)/P(B) considering the
;;      largest possible intersection between B and C
;;
;;         sBC <= min(1, sC/sB)
;;
;; Calculation:
;;
;;   sAC = [sAB.sBC + ((1-sAB)(sC - sBsBC))/(1-sB)]
;;
;; To avoid division by 0 (which the preconditions do not necessarily
;; prevent), i.e. when sB tends to 1 we find that
;;
;;   sAC tends to sC
;;
;; Indeed, in such case sAB tends to sB and sBC tends to sC. As a
;; result (according to the main formula above) sAC tends to
;;
;;   sB.sC + ((1-sB)(sC - sBsC))/(1-sB) = sB.sC + (sC - sB.sC) = sC
;;
;; If the preconditions are not met it returns sAC = 0, although
;; honnestly the deduction rule should not have been applied in the
;; first place.
;; -----------------------------------------------------------------------------


;; Limit a number to be within a certain range
(define (limit x l u)
  (max l (min u x)))

; Consistency Conditions
(define (smallest-intersection-probability sA sB)
  (limit (/ (+ sA sB -1) sA) 0 1))

(define (largest-intersection-probability sA sB)
  (limit (/ sB sA) 0 1))

(define (conditional-probability-consistency sA sB sAB)
  (and (< 0 sA)
       (<= (smallest-intersection-probability sA sB) sAB)
       (<= sAB (largest-intersection-probability sA sB))))

; Main Formula

(define (simple-deduction-strength-formula sA sB sC sAB sBC)
  (if
     (and
        (conditional-probability-consistency sA sB sAB)
        (conditional-probability-consistency sB sC sBC))
     ;; Preconditions are met
     (if (< 0.99 sB)
        ;; sB tends to 1
        sC
        ;; otherwise
        (+ (* sAB sBC) (/ (* (- 1 sAB) (- sC (* sB sBC))) (- 1 sB))))
     ;; Preconditions are not met
     0))

;; =============================================================================
;; Inversion Formula
;;
;; Preconditions:
;;
;;   1. Check that P(B|A) makes sense
;;
;;      1.a. P(B|A) is defined
;;
;;         0.0 < sA
;;
;;      1.b. P(B|A) is greater than or equal to P(A,B)/P(A) considering
;;      the smallest possible intersection between A and B
;;
;;         max((sA+sB-1)/sA, 0) <= sAB
;;
;;      1.c. P(B|A) is smaller than P(A,B)/P(A) considering the largest
;;      possible intersection between A and B
;;
;;         sAB <= min(1, sB/sA)
;;
;;   2. Check that P(A|B) makes sense
;;
;;      1. P(A|B) is defined
;;
;;         0.0 < sB
;;
;;         And that is all, ultimately the calculation should result
;;         into a consistent sBA (or let's hope so).
;;
;; Calculation:
;;
;;   sBA = (sAB * sB) / sA
;;
;; -----------------------------------------------------------------------------

(define (inversion-consistency sA sB sAB)
  (and (< 0 sA)
       (< 0 sB)
       (<= (smallest-intersection-probability sA sB) sAB)
       (<= sAB (largest-intersection-probability sA sB))))

(define (inversion-strength-formula sA sB sAB)
  (if (inversion-consistency sA sB sAB)
      (/ (* sAB sA) sB)
      0))

;; =============================================================================
;; Invert formula
;;
;; Inverts the input
;; -----------------------------------------------------------------------------

(define (invert a)
    (/ 1.0 a))

;; =============================================================================
;; Negate formula
;;
;; Negates the probability
;; -----------------------------------------------------------------------------

(define (negate a)
    (- 1 a))

;; =============================================================================
;; Transitive Similarity Formula
;;
;; Returns the strength value of the transitive similarity rule
;; -----------------------------------------------------------------------------

(define (transitive-similarity-strength-formula sA sB sC sAB sBC )
    (let
        ((T1 (/ (* (+ 1 (/ sB sA)) (sAB)) (+ 1 sAB)))
         (T2 (/ (* (+ 1 (/ sC sB)) (sBC)) (+ 1 sBC)))
         (T3 (/ (* (+ 1 (/ sB sC)) (sBC)) (+ 1 sBC)))
         (T4 (/ (* (+ 1 (/ sA sB)) (sAB)) (+ 1 sAB))))
        (invert (- (+ (invert (+ (* T1 T2) (* (negate T1) (/ (- sC (* sB T2)) (negate sB)))))
                      (invert (+ (* T3 T4) (* (negate T3) (/ (- sC (* sB T4)) (negate sB)))))) 1))))

;; =============================================================================
;; PreciseModusPonens formula
;;
;; Returns the strength value of the precise modus ponens rule
;; -----------------------------------------------------------------------------

(define (precise-modus-ponens-strength-formula sA sAB snotAB)
    (+ (* sAB sA) (* snotAB (negate sA))))


#<unspecified>

**_load the code of deduction.scm_**

In [3]:
;; =============================================================================
;; DeductionRule
;;
;; <LinkType>
;;   A
;;   B
;; <LinkType>
;;   B
;;   C
;; |-
;; <LinkType>
;;   A
;;   C
;;
;; Due to type system limitations, the rule has been divided into 3:
;;       deduction-inheritance-rule
;;       deduction-implication-rule
;;       deduction-subset-rule
;;
;; This deduction rule makes assumptions to avoid having too many
;; premises. Another more precise rule should be created as well.
;;
;; -----------------------------------------------------------------------------

;; Generate the corresponding deduction rule given its link-type and
;; the type for each variable (the same for all 3).
(define (gen-deduction-rule link-type var-type)
  (let* ((A (Variable "$A"))
         (B (Variable "$B"))
         (C (Variable "$C"))
         (AB (link-type A B))
         (BC (link-type B C))
         (AC (link-type A C)))
    (Bind
      (VariableList
        (TypedVariable A var-type)
        (TypedVariable B var-type)
        (TypedVariable C var-type))
      (And
        AB
        BC
        (Not (Identical A C)))
      (ExecutionOutput
        (GroundedSchema "scm: deduction-formula")
        (List
          ;; Conclusion
          AC
          ;; Premises
          ;;
          ;; TODO: perhaps A, B, C should be added as premises as
          ;; they are used in the formula.
          AB
          BC)))))

(define deduction-inheritance-rule
  (let ((var-type (TypeChoice
                    (TypeNode "ConceptNode")
                    (TypeNode "AndLink")
                    (TypeNode "OrLink")
                    (TypeNode "NotLink"))))
    (gen-deduction-rule InheritanceLink var-type)))

(define deduction-implication-rule
  (let ((var-type (TypeChoice
                    (TypeNode "PredicateNode")
                    (TypeNode "LambdaLink")
                    (TypeNode "AndLink")
                    (TypeNode "OrLink")
                    (TypeNode "NotLink"))))
    (gen-deduction-rule ImplicationLink var-type)))

(define deduction-subset-rule
  (let ((var-type (TypeChoice
                    (TypeNode "ConceptNode")
                    (TypeNode "AndLink")
                    (TypeNode "OrLink")
                    (TypeNode "NotLink"))))
    (gen-deduction-rule SubsetLink var-type)))

(define (deduction-formula conclusion . premises)
  (if (= (length premises) 2)
    (let*
        ((AC conclusion)
         (AB (list-ref premises 0))
         (BC (list-ref premises 1))
         (sA (cog-stv-strength (gar AB)))
         (cA (cog-stv-confidence (gar AB)))
         (sB (cog-stv-strength (gar BC)))
         (cB (cog-stv-confidence (gar BC)))
         (sC (cog-stv-strength (gdr BC)))
         (cC (cog-stv-confidence (gdr BC)))
         (sAB (cog-stv-strength AB))
         (cAB (cog-stv-confidence AB))
         (sBC (cog-stv-strength BC))
         (cBC (cog-stv-confidence BC))
         (alpha 0.9) ; how much confidence is lost at each deduction step

         ;; Hacks to overcome the lack of distributional TV. If s=1
         ;; and c=0, then assign s to the mode value satisfying the
         ;; deduction consistency constraint (what a pain, let's use
         ;; 0.25 for now).
         (sA (if (and (< 0.99 sA) (<= cA 0)) 0.25 sA))
         (sB (if (and (< 0.99 sB) (<= cB 0)) 0.25 sB))
         (sC (if (and (< 0.99 sC) (<= cC 0)) 0.25 sC)))
      (if (and
           (or (= 0 cA) (= 0 cB) (= 0 cAB)
               (conditional-probability-consistency sA sB sAB))
           (or (= 0 cB) (= 0 cC) (= 0 cBC)
               (conditional-probability-consistency sB sC sBC)))
          (if (< 0.99 (* sB cB))
              ;; Hack to overcome for the lack of distributional
              ;; TV. This covers the case where B fully confidently
              ;; tends to 1. See formulas.scm Simple Deduction
              ;; Formula comment for more explanations. This
              ;; overlaps with the implication-introduction-rule.
              (let ((sAC sC)
                    (cAC (* alpha cA cC)))
                (if (and (< 1e-8 sAC) (< 1e-8 cAC)) ;; Don't create zero
                                              ;; knowledge. Note that
                                              ;; sAC == 0 is not zero
                                              ;; knowledge but it's
                                              ;; annoying in the
                                              ;; current hacky
                                              ;; situation.
                    (cog-merge-hi-conf-tv! AC (stv sAC cAC))))
              ;; Branch if sB * cB <= 0.99
              (let* ((sAC (if (or (< 0.99 (* sAB cAB)) (< 0.99 (* sBC cBC)))
                              ;; Hack to overcome for the lack of
                              ;; distributional TV. This covers the case
                              ;; where little is known about A and B
                              ;; (i.e. their strength is meaningless), yet
                              ;; we can confidently calculate sAC because
                              ;; sAB and sBC are so high anyway.
                              (* sAB sBC)
                              ;; Otherwise fall back on the naive formula
                              (simple-deduction-strength-formula sA sB sC sAB sBC)))
                     (cAC (min cAB cBC))
                     ;; Unless the 2 implication are fully confident
                     ;; decrease the confidence by some factor. I'm not
                     ;; sure how justify this for now, it's perhaps a
                     ;; bad hack.
                     (cAC (* (if (< cAC 0.99) alpha 1.0) cAC)))
                (if (and (< 1e-8 sAC) (< 1e-8 cAC)) ;; Don't create zero
                                              ;; knowledge. Note that
                                              ;; sAC == 0 is not zero
                                              ;; knowledge but it's
                                              ;; annoying in the
                                              ;; current hacky
                                              ;; situation.
                    (cog-merge-hi-conf-tv! AC (stv sAC cAC)))))))))

;; Name the rules
(define deduction-inheritance-rule-name
  (DefinedSchemaNode "deduction-inheritance-rule"))
(DefineLink deduction-inheritance-rule-name
  deduction-inheritance-rule)

(define deduction-implication-rule-name
  (DefinedSchemaNode "deduction-implication-rule"))
(DefineLink deduction-implication-rule-name
  deduction-implication-rule)

(define deduction-subset-rule-name
  (DefinedSchemaNode "deduction-subset-rule"))
(DefineLink deduction-subset-rule-name
  deduction-subset-rule)

(DefineLink
   (DefinedSchemaNode "deduction-subset-rule")
   (BindLink
      (VariableList
         (TypedVariableLink
            (VariableNode "$A")
            (TypeChoice
               (TypeNode "ConceptNode")
               (TypeNode "AndLink")
               (TypeNode "OrLink")
               (TypeNode "NotLink")
            )
         )
         (TypedVariableLink
            (VariableNode "$B")
            (TypeChoice
               (TypeNode "ConceptNode")
               (TypeNode "AndLink")
               (TypeNode "OrLink")
               (TypeNode "NotLink")
            )
         )
         (TypedVariableLink
            (VariableNode "$C")
            (TypeChoice
               (TypeNode "ConceptNode")
               (TypeNode "AndLink")
               (TypeNode "OrLink")
               (TypeNode "NotLink")
            )
         )
      )
      (AndLink
         (SubsetLink
            (VariableNode "$B")
            (VariableNode "$C")
         )
         (SubsetLink


**_load the code of simple-assertions.scm_**

In [4]:
; Define the concepts

(ConceptNode "Socrates" (stv .001 0.9))
(ConceptNode "Einstein" (stv .001 0.9))
(ConceptNode "Peirce" (stv .001 0.9))
(ConceptNode "man" (stv .01 0.9))
(ConceptNode "human" (stv .02 0.9))

; Define the instances of man

(InheritanceLink (stv 0.9 0.9)
    (ConceptNode "Socrates")
    (ConceptNode "man"))

(InheritanceLink (stv 0.9 0.9)
    (ConceptNode "Einstein")
    (ConceptNode "man"))

(InheritanceLink (stv 0.9 0.9)
    (ConceptNode "Peirce")
    (ConceptNode "man"))

; Define what man is part of

(InheritanceLink (stv 0.9 0.9)
    (ConceptNode "man")
    (ConceptNode "human"))

; Assign some additional memberships as well

(InheritanceLink (stv 0.9 0.9)
    (ConceptNode "Einstein")
    (ConceptNode "violin-players"))

; Pattern to match to check the output

(define find-humans
    (BindLink
        (VariableNode "$X")
        (InheritanceLink
            (VariableNode "$X")
            (ConceptNode "human"))
        (ListLink
            (VariableNode "$X"))))

#<unspecified>

In [5]:
;query to find all humans using 
;the 'find-humans' method is defined in the code for simple-assertions.scm
(cog-execute! find-humans)

(SetLink
   (ListLink
      (ConceptNode "man" (stv 0.01 0.9))
   )
)


Observe that there is only one instance of human defined.

Output : (SetLink
   (ListLink
      (ConceptNode "man" (stv 0.01 0.9))
   )
)

In [6]:
;query to deduces new facts about relationships
;The command below creates new relationships (i.e It deduces new facts about relationships.)
(cog-execute! deduction-inheritance-rule)

(SetLink
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Einstein" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Socrates" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Peirce" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
)


You can see from the output that it deduced that there were three other humans

Output : (SetLink
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Einstein" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Socrates" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Peirce" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
)

In [7]:
;query to find all humans
(cog-execute! find-humans)

(SetLink
   (ListLink
      (ConceptNode "Socrates" (stv 0.001 0.9))
   )
   (ListLink
      (ConceptNode "Einstein" (stv 0.001 0.9))
   )
   (ListLink
      (ConceptNode "man" (stv 0.01 0.9))
   )
   (ListLink
      (ConceptNode "Peirce" (stv 0.001 0.9))
   )
)


Observe that there are now 3 additional instances of human defined 

Output : (SetLink
   (ListLink
      (ConceptNode "Socrates" (stv 0.001 0.9))
   )
   (ListLink
      (ConceptNode "Einstein" (stv 0.001 0.9))
   )
   (ListLink
      (ConceptNode "man" (stv 0.01 0.9))
   )
   (ListLink
      (ConceptNode "Peirce" (stv 0.001 0.9))
   )
)

# For scheme Environment

**Note: we assume you are starting in your home directory - to get there type :**

> *$ cd ~*

**From the command prompt, enter the scheme environment by typing : **

> *$ guile*

** _load the required modules_ **<br>

> ``scheme@(guile-user)> (use-modules (ice-9 readline))``

> ``scheme@(guile-user)> (activate-readline)``

> ``scheme@(guile-user)> (add-to-load-path "/usr/local/share/opencog/scm")``

> ``scheme@(guile-user)> (add-to-load-path ".")``

> ``scheme@(guile-user)> (use-modules (opencog))``

> ``scheme@(guile-user)> (use-modules (opencog query))``

> ``scheme@(guile-user)> (use-modules (opencog exec))``

> ``scheme@(guile-user)> (use-modules (opencog logger))``

**Note: _In the following two code blocks, you may have to change the paths to suit your installation._**<br>

** _load the deduction rule definitions_** <br>
> ``scheme@(guile-user)> (load "./opencog/opencog/pln/rules/term/deduction.scm")``

** _load the simple-assertion.scm_**<br>
> ``scheme@(guile-user)> (load "./opencog/tests/pln/rules/simple-assertions.scm")``

** _query to find all humans_ **<br>
> ``scheme@(guile-user)> (cog-execute! find-humans)``

**Observe that there is only one instance of human defined.**<br> 
** Output :**
$2 = (SetLink
   (ListLink
      (ConceptNode "man" (stv 0.01 0.9))
   )
)
**

** The command **_(cog-execute! deduction-inheritance-rule)_** creates new relationships - deduces new facts about relationships. **<br>
> ``scheme@(guile-user)> (cog-execute! deduction-inheritance-rule)``

** You can see from the output that it deduced that there were three other humans **<br>
** Output : **
$3 = (SetLink
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Einstein" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Socrates" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
   (InheritanceLink (stv 0.81111111 0.81)
      (ConceptNode "Peirce" (stv 0.001 0.9))
      (ConceptNode "human" (stv 0.02 0.9))
   )
)

** _query to find all humans_ **<br>
> ``scheme@(guile-user)> (cog-execute! find-humans)``

** Observe that there are now 3 additional instances of human defined **<br>
** Output :**
$4 = (SetLink
   (ListLink
      (ConceptNode "Socrates" (stv 0.001 0.9))
   )
   (ListLink
      (ConceptNode "Einstein" (stv 0.001 0.9))
   )
   (ListLink
      (ConceptNode "man" (stv 0.01 0.9))
   )
   (ListLink
      (ConceptNode "Peirce" (stv 0.001 0.9))
   )
)

### additional practice
Create new ConceptNodes yourself with inheritance from human.<br>
For example :
> (ConceptNode "Adam Ford" (stv .001 0.9))<br>
  (Inheritance (stv .9 .9) (ConceptNode "Adam Ford") (ConceptNode "man"))<br>
  **_Note : stv stands for SimpleTruthValue_**

**Note :** The PLN deduction rule is probabilistic and will check whether the stv values on the ConceptNodes and InheritanceLinks are consistent with probability theory. It will silently discard inferences if they're not consistent, see [1](https://github.com/opencog/opencog/blob/master/opencog/pln/rules/formulas.scm#L13) for more details. Likewise, the confidence, second value of the stv, should be strictly positive; otherwise, the deduction will likely not be able to infer anything useful.

Try to find-humans before deducing inheritances :
> (cog-bind find-humans)<br>
**_Note : the new ConceptNode shouldn't be displayed yet._**


Next deduce the inheritances : 
> (cog-bind deduction-inheritance-rule)

Now find the humans again : 
> (cog-bind find-humans)<br> 
**_Note : Now 'Adam Ford' should be listed as one of the humans._**


**_Did you notice above that we set up an inheritance link such that 'Adam Ford' inherits from 'man'? Why didn't we just inherit from human?_**<br>

Create a new ConceptNode 'Ben Goertzel', the same way as we did above for 'Adam Ford', with Inheritance link to 'human' : 
> (Inheritance (stv .9 .9) (ConceptNode "Ben Goertzel") (ConceptNode "human"))


Next deduce the inheritances (as above).<br>
What do you expect the output of _(cog-bind deduction-inheritance-rule)_ will be?
Does "Ben Goertzel" appear...? why?

# More PLN Examples
** A deduction-rule example is given here in your local opencog repository:**
* ~/opencog/examples/pln

**deduction examples in python: ** 
* ./opencog/opencog/python/pln_old/examples/deduction/

** Old Python 'Smokes' example:**
* ./opencog/opencog/python/pln_old/examples/tuffy/smokes

**Add an example stepping through some PLN inferences by hand in the Scheme shell, applying each rule by hand instead of using a chainer. The smokes example may be good for this.**<br>
[PLN rules on git](https://github.com/opencog/opencog/tree/master/opencog/pln/rules)